# Automatic differentiation

Consider the function
$$
f(x) = \cos(x)\sin(x)
$$

When we want to evaluate the function numerically at a specific $x$, say $x=1$ we can implement a computer program like

~~~
x = 1
f = cos(x)*sin(x)
~~~

Now suppose we need the derivative as well. For this example, it is a simple exercise to calculate the derivative as 
$$
f'(x) = \cos(x)\cos(x) -\sin(x)\sin(x) = \cos(x)^2 - \sin(x)^2
$$

~~~
x = 1
df_dx =  cos(x)*cos(x) - sin(x)*sin(x)
~~~

But could we have calculated the derivative without coding it up explicitely, that is without symbolically evaluating it a priori by hand?
The answer turns out to be yes and it is a quite fascinating subject called __automatic differentiation__. Interestingly, this algorithm, known also as __backpropagation__, is in the core of todays artificial intelligence systems, programs that learn how to program themselves from input and output examples. See https://www.youtube.com/watch?v=aircAruvnKk for an introduction to a particular type of model, known as a __neural network__.

To symbolically evaluate the derivative, we can use the chain rule which says when 
$$
f(x) = g(h(x))
$$
we have
$$
f'(x) = g'(h(x)) h'(x)
$$

When we have functions of two variables, such as
$$
f(x) = g(h_1(x), h_2(x)) 
$$
we have to adopt the partial derivative notation
$$
\frac{\partial f}{\partial x}  = \frac{\partial f}{\partial g} ( \frac{\partial g}{\partial h_1} \frac{\partial h_1}{\partial x} +   \frac{\partial g}{\partial h_2} \frac{\partial h_2}{\partial x})
$$


To see a concrete example, consider the function
$$
f(x) = \sin(x)\cos(x)
$$


We define 
\begin{align}
h_1(x) & = c = \cos(x) \\
h_2(x) & = s = \sin(x) \\
g(c,s) & = g = c \times s \\
f & = g(c,s)
\end{align}

that is equivalent to the following program
~~~
x = 1
c = cos(x)
s = sin(x)
g = c * s
f = g
~~~

This program can be represented by the following directed computation graph:
<img src="../latex_figures/cos_sin.png" width="300">

The function can be evaluated by traversing the variable nodes of the directed graph from the inputs to the outputs in the topological order. At each variable node, we merely evaluate the incoming function. Topological order guarantees that the inputs for the function are already calculated. 

It is not obvious, but the derivatives can also be calculated easily. By the chain rule, we have 
\begin{eqnarray}
\frac{\partial f}{\partial x} &=& \frac{\partial f}{\partial g} \frac{\partial g}{\partial c} \frac{\partial c}{\partial x} +  \frac{\partial f}{\partial g} \frac{\partial g}{\partial s} \frac{\partial s}{\partial x} \\
&=& 1 \cdot s \cdot \sin(x) +  1 \cdot c \cdot (-\cos(x)) \\
&=& 1 \cdot \sin(x) \cdot \sin(x) +  1 \cdot \cos(x) \cdot (-\cos(x)) \\
\end{eqnarray}

The derivative could have been calculated numerically by the following program

~~~
df_dx = 0, df_ds = 0, df_dc = 0, df_dg = 0 
df_df = 1

df_dg +=  df_df           // df/dg = 1
df_dc +=  s * df_dg       // dg/dc = s
df_ds +=  c * df_dg       // dg/ds = c
df_dx +=  cos(x) * df_ds  // ds/dx = cos(x)
df_dx += -sin(x) * df_dc  // dc/dx = -sin(x)
~~~

Note that the total derivative consists of sums of several terms. Each term is the product of the derivatives along the path leading from $f$ to $x$. In the above example, there are only two paths: 

- $f,g,c,x$
- $f,g,s,x$

$$
\frac{\partial f}{\partial x}  = \frac{\partial f}{\partial g} \frac{\partial g}{\partial c} \frac{\partial c}{\partial x} +  \frac{\partial f}{\partial g} \frac{\partial g}{\partial s} \frac{\partial s}{\partial x}
$$

It is not obvious in this simple example but the fact that we are propagating backwards makes us save computation by storing the intermediate variables.

This program can be represented by the following directed computation graph:
<img src="../latex_figures/cos_sin_with_df.png" width="400">

Note that during the backward pass, if we traverse variable nodes in the reverse topological order, we only need the derivatives already computed and perhaps some of the variables that are connected to the function node that are computed during the forward pass. As an example, consider

$$
\frac{\partial f}{\partial c} = \frac{\partial f}{\partial g} \frac{\partial g}{\partial c}
$$
The first term is already available during the backward pass. The second term needs to be programmed by calculating the partial derivative of $g(s,c) = sc$ with respect to $c$. It has a simple form, namely $s$. More importantly, the numerical value is also immediately available, as it is calculated during the forward pass. For each function type, this calculation will be different but is nevertheless straightforward for all basic functions, including the binary arithmetic operators $+,-,\times$ and $\div$.


# Programming Assignment

In this exercise, you will write a program that gets two input files.  The first file will be the function definition file, the second will be the input values for which the function and its derivative is to be calculated

## Function Definition File
The function definition file will have the following format:
~~~
<inputs>
<output>
<assignments>
~~~

An example is the following for the function
$$
f(x_1, x_2)  =  \sin(2x_1)\cos(x_1 x_2)
$$

~~~
input x_1
input x_2
output f
t_0 = mult 2 x_1
t_1 = sin  t_0
t_2 = mult x_1 x_2
t_3 = cos t_2
t_4 = mult t_1 t_3
f   = t_4 
~~~

## Input Values File

~~~
<input 1><input 2>...<input n>
<value 1><value 2>...<value n>
<value 1><value 2>...<value n>
<value 1><value 2>...<value n>
...
~~~

If we want to evaluate the function on values $(x_1,x_2) = \{(0,0),(1.2,-3),(5,5)\}$, the example file format will be
~~~
x_1 x_2
0 0
1.2 -3
5 5
~~~

Your program will have two output files: Function values and the derivatives (upto 5 digits of precision)

~~~
f
0.0
-0.60573
-0.53924
~~~

The partial derivatives are

$$
\frac{\partial f}{\partial x_1}  =  2\cos(2x_1)\cos(x_1 x_2) - \sin(2x_1)\sin(x_1 x_2) x_2
$$

$$
\frac{\partial f}{\partial x_2}  =  -\sin(2x_1)\sin(x_1 x_2)x_1 
$$


Hence, the output file will be
~~~
df/dx_1 df/dx_2
2.0 -0.0
0.24682 -0.35869
0.20232 -0.36001
~~~

The basic binary arithmetic operations
~~~
add
subs
mult
divide
~~~

The following functions that are a subset of the cmath library must also be supported
~~~
cos
sin
tan
acos
asin
atan
exp
log
log10
pow
sqrt
~~~


The algorithm can be sketched as follows:

- Construct a directed graph of variable and function nodes
- Check if the graph is acyclic. If cyclic, generate an error message. The function is invalid
- Forward pass: Evaluate the function graph in topological order to find the temporary variables. Write the function output.
- Backward pass: Evaluate the derivatives by initializing $\partial f/\partial f = 1$ and propagate in the reverse topological order for finding the derivatives. To implement the chain rule, proceed by multiplication of the derivative at the output variable with the partial derivative across the immediate neighbors of a function node.




